# Census Data Tools

Initiate and configure the logger. This will save the outputs of the log to the specified file. 

In [2]:
from morpc.logs import config_logs

config_logs('./temp_data/morpc-census-demo.log', level='info') ## Designate the logger level. 

2025-11-12 11:55:35,455 | INFO | morpc.logs.config_logs: Set up logging save to file ./temp_data/morpc-census-demo.log


## morpc.census.api

This module has been developed to fetch and save Census data from various API endpoints. 

## Census API Class

The class CensusAPI class does most of the heavy lifting, formulating the API call, getting the data, wrangling it into a long format, creating frictionless schema and resource files, and saving the data. 

The class takes the following parameters:

1. The "survey/table" to get the data from. 
2. The year of the data. 
3. The group of variables that you want to call.
4. The geographic "scope" of the data you want to get the data for.

Optionally:

5. The geographic scale of the data you want to get the data for.
6. A subset of the variables in the chosen group.



In [3]:
# TODO: Timeseries calls.  

# TODO: Getting select variables from various groups.

## Example: Compare divorced population for all tracts in 15-County region.

### 1. See currently implemented endpoints for "survey/table" parameter.

Use the IMPLEMENTED_ENDPOINTS constant to see currently available options.

In [4]:
from morpc.census.api import IMPLEMENTED_ENDPOINTS

IMPLEMENTED_ENDPOINTS

2025-11-12 11:55:35,478 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data with parameters None.


['acs/acs1',
 'acs/acs1/profile',
 'acs/acs1/subject',
 'acs/acs5',
 'acs/acs5/profile',
 'acs/acs5/subject',
 'dec/pl',
 'dec/dhc',
 'dec/ddhca',
 'dec/ddhcb',
 'dec/sf1',
 'dec/sf2',
 'dec/sf3']

### 2. See available vintages

To see all available endpoints provided by the Census Bureau and their available vintages, see ALL_AVAIL_ENPOINTS

In [5]:
from morpc.census.api import ALL_AVAIL_ENDPOINTS 

ALL_AVAIL_ENDPOINTS['acs/acs5']

[2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2009,
 2019,
 2020,
 2021,
 2022,
 2023]

### 3. See available variable groups for a survey/table.

In [6]:
from morpc.census.api import get_table_groups

{k: v['description'] for k, v in get_table_groups('acs/acs5', 2023).items()}

# TODO: Hierarchical lookup for variables of interest.

2025-11-12 11:55:36,809 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2023/acs/acs5/groups.json with parameters None.


{'B01001': 'Sex by Age',
 'B01001A': 'Sex by Age (White Alone)',
 'B01001B': 'Sex by Age (Black or African American Alone)',
 'B01001C': 'Sex by Age (American Indian and Alaska Native Alone)',
 'B01001D': 'Sex by Age (Asian Alone)',
 'B01001E': 'Sex by Age (Native Hawaiian and Other Pacific Islander Alone)',
 'B01001F': 'Sex by Age (Some Other Race Alone)',
 'B01001G': 'Sex by Age (Two or More Races)',
 'B01001H': 'Sex by Age (White Alone, Not Hispanic or Latino)',
 'B01001I': 'Sex by Age (Hispanic or Latino)',
 'B01002': 'Median Age by Sex',
 'B01002A': 'Median Age by Sex (White Alone)',
 'B01002B': 'Median Age by Sex (Black or African American Alone)',
 'B01002C': 'Median Age by Sex (American Indian and Alaska Native Alone)',
 'B01002D': 'Median Age by Sex (Asian Alone)',
 'B01002E': 'Median Age by Sex (Native Hawaiian and Other Pacific Islander Alone)',
 'B01002F': 'Median Age by Sex (Some Other Race Alone)',
 'B01002G': 'Median Age by Sex (Two or More Races)',
 'B01002H': 'Median A

###  4. Select the geographic scope of the data.

To see currently implemented scopes. See morpc.census.geos.SCOPES

In [7]:
from morpc.census.geos import SCOPES

SCOPES

# TODO: Passing list of geoids to ucgid parameter in api call

{'us': {'for': 'us:1'},
 'region15': {'in': 'state:39',
  'for': 'county:041,045,049,089,097,129,159,083,101,117,047,073,091,127,141'},
 'region10': {'in': 'state:39',
  'for': 'county:041,045,049,089,097,129,159,083,101,117'},
 'region7': {'in': 'state:39', 'for': 'county:041,045,049,089,097,129,159'},
 'region-corpo': {'in': 'state:39',
  'for': 'county:045,083,097,101,117,129,159'},
 'alabama': {'for': 'state:01'},
 'alaska': {'for': 'state:02'},
 'arizona': {'for': 'state:04'},
 'arkansas': {'for': 'state:05'},
 'california': {'for': 'state:06'},
 'colorado': {'for': 'state:08'},
 'connecticut': {'for': 'state:09'},
 'delaware': {'in': 'state:39', 'for': 'county:041'},
 'district of columbia': {'for': 'state:11'},
 'florida': {'for': 'state:12'},
 'georgia': {'for': 'state:13'},
 'hawaii': {'for': 'state:15'},
 'idaho': {'for': 'state:16'},
 'illinois': {'for': 'state:17'},
 'indiana': {'for': 'state:18'},
 'iowa': {'for': 'state:19'},
 'kansas': {'for': 'state:20'},
 'kentucky': {

### (Optional) 5. Select a smaller geographic scale to get data for. 

For example if you want to get all the tracts in the 15-county MORPC region.

To see available combinations of scope and scale see morpc.census.geos.PSUEDOS.

The key is the parent sumlevel and the values represent the child sumlevel to get the data for.

In [8]:
from morpc.census.geos import PSEUDOS

PSEUDOS

## All available scale levels if the sumlevel of the "scope" paramter is Counties (050)

PSEUDOS['050']

['0600000',
 '06V0000',
 '1000000',
 '1400000',
 '1500000',
 '1600000',
 '7000000',
 '8600000',
 '8710000']

### (Optional) 6. A list of variables to get from the group.

In [9]:
from morpc.census.api import get_group_variables

{k: v['label'] for k, v in get_group_variables('acs/acs5', 2023, 'B12001').items()}

2025-11-12 11:55:37,240 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2023/acs/acs5/groups/B12001.json with parameters None.


{'B12001_001E': 'Estimate!!Total:',
 'B12001_001EA': 'Annotation of Estimate!!Total:',
 'B12001_001M': 'Margin of Error!!Total:',
 'B12001_001MA': 'Annotation of Margin of Error!!Total:',
 'B12001_002E': 'Estimate!!Total:!!Male:',
 'B12001_002EA': 'Annotation of Estimate!!Total:!!Male:',
 'B12001_002M': 'Margin of Error!!Total:!!Male:',
 'B12001_002MA': 'Annotation of Margin of Error!!Total:!!Male:',
 'B12001_003E': 'Estimate!!Total:!!Male:!!Never married',
 'B12001_003EA': 'Annotation of Estimate!!Total:!!Male:!!Never married',
 'B12001_003M': 'Margin of Error!!Total:!!Male:!!Never married',
 'B12001_003MA': 'Annotation of Margin of Error!!Total:!!Male:!!Never married',
 'B12001_004E': 'Estimate!!Total:!!Male:!!Now married:',
 'B12001_004EA': 'Annotation of Estimate!!Total:!!Male:!!Now married:',
 'B12001_004M': 'Margin of Error!!Total:!!Male:!!Now married:',
 'B12001_004MA': 'Annotation of Margin of Error!!Total:!!Male:!!Now married:',
 'B12001_005E': 'Estimate!!Total:!!Male:!!Now ma

In [10]:
variables =['B12001_001E', 'B12001_002E', 'B12001_010E', 'B12001_011E', 'B12001_019E']

### Get the data by calling CensusAPI

In [11]:
from morpc.census.api import CensusAPI

b12001_2023 = CensusAPI('acs/acs5', 2023, 'B12001', 'region15', scale = 'tract', variables=variables)

2025-11-12 11:55:37,508 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.__init__: Initializing CENSUS_API_DATA object for census-acs-acs5-2023-tract-region15-b12001.
2025-11-12 11:55:37,509 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2023/acs/acs5/groups.json with parameters None.
2025-11-12 11:55:37,880 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.validate: Validating selected parameters
2025-11-12 11:55:37,881 | INFO | morpc.census.api.valid_survey_table: acs/acs5 is valid and implemented.
2025-11-12 11:55:37,881 | INFO | morpc.census.api.valid_vintage: 2023 is valid vintage for acs/acs5
2025-11-12 11:55:37,882 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2023/acs/acs5/groups.json with parameters None.
2025-11-12 11:55:38,223 | INFO | morpc.census.api.valid_group: Group B12001 valid group for 2023 acs/acs5.
2025-11-12 11:55:38,225 | INFO | morpc.

The raw data is available and stored in the class as .DATA

In [12]:
b12001_2023.DATA

,GEO_ID,NAME,B12001_001E,B12001_002E,B12001_010E,B12001_011E,B12001_019E
0,1400000US39041010100,Census Tract 101; Delaware County; Ohio,4670,2370,170,2300,240
1,1400000US39041010200,Census Tract 102; Delaware County; Ohio,5073,2596,385,2477,463
2,1400000US39041010420,Census Tract 104.20; Delaware County; Ohio,2443,1240,144,1203,212
3,1400000US39041010421,Census Tract 104.21; Delaware County; Ohio,2240,1062,81,1178,145
4,1400000US39041010422,Census Tract 104.22; Delaware County; Ohio,4845,2355,44,2490,290
...,...,...,...,...,...,...,...
578,1400000US39159050502,Census Tract 505.02; Union County; Ohio,1959,928,132,1031,179
579,1400000US39159050601,Census Tract 506.01; Union County; Ohio,8524,4337,188,4187,197
580,1400000US39159050602,Census Tract 506.02; Union County; Ohio,3654,1995,156,1659,188
581,1400000US39159050701,Census Tract 507.01; Union County; Ohio,3479,1662,70,1817,72


The class also automatically creates a long table format which is the format we will save the data as. 

In [13]:
b12001_2023.LONG

,GEO_ID,NAME,reference_period,variable_label,variable,estimate
0,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:,B12001_001,4670
3,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:!!Male:,B12001_002,2370
4,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:!!Male:!!Divorced,B12001_010,170
1,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:!!Female:,B12001_011,2300
2,1400000US39041010100,Census Tract 101; Delaware County; Ohio,2023,Total:!!Female:!!Divorced,B12001_019,240
...,...,...,...,...,...,...
2910,1400000US39159050702,Census Tract 507.02; Union County; Ohio,2023,Total:,B12001_001,2494
2913,1400000US39159050702,Census Tract 507.02; Union County; Ohio,2023,Total:!!Male:,B12001_002,1197
2914,1400000US39159050702,Census Tract 507.02; Union County; Ohio,2023,Total:!!Male:!!Divorced,B12001_010,50
2911,1400000US39159050702,Census Tract 507.02; Union County; Ohio,2023,Total:!!Female:,B12001_011,1297


#### morpc.census.api.DimensionTable

The other class that is implemented is the dimension table. These are typically used for situations in which you want the data to be human readable. 

This is designed to replicate how the Census often represents it's data with the different levels of the variable group as rows and the geography and years as columns. 

In [14]:
from morpc.census.api import DimensionTable

dim_table = DimensionTable(b12001_2023.LONG)

2025-11-12 11:55:40,247 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:40.247656.__init__: Initializing DIMENSION_TABLE object.


In [15]:
dim_table.wide().T

2025-11-12 11:55:40,262 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:40.247656.wide: Pivoting data into wide format.
2025-11-12 11:55:40,263 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:40.247656.create_description_table: Creating description table from variable labels.


0                                                                                Total:  \
1                                                                                         
2                                                                                         
GEO_ID               NAME                                       reference_period          
1400000US39041010100 Census Tract 101; Delaware County; Ohio    2023               4670   
1400000US39041010200 Census Tract 102; Delaware County; Ohio    2023               5073   
1400000US39041010420 Census Tract 104.20; Delaware County; Ohio 2023               2443   
1400000US39041010421 Census Tract 104.21; Delaware County; Ohio 2023               2240   
1400000US39041010422 Census Tract 104.22; Delaware County; Ohio 2023               4845   
...                                                                                 ...   
1400000US39159050502 Census Tract 505.02; Union County; Ohio    2023               1959   
1400000US39159050601 Census Tract 506.01; Union County; Ohio    2023               8524   
1400000US39159050602 Census Tract 506.02; Union County; Ohio    2023               3654   
1400000US39159050701 Census Tract 507.01; Union County; Ohio    2023               3479   
1400000US39159050702 Census Tract 507.02; Union County; Ohio    2023               2494   

0                                                                                       \
1                                                                                Male:   
2                                                                                        
GEO_ID               NAME                                       reference_period         
1400000US39041010100 Census Tract 101; Delaware County; Ohio    2023              2370   
1400000US39041010200 Census Tract 102; Delaware County; Ohio    2023              2596   
1400000US39041010420 Census Tract 104.20; Delaware County; Ohio 2023              1240   
1400000US39041010421 Census Tract 104.21; Delaware County; Ohio 2023              1062   
1400000US39041010422 Census Tract 104.22; Delaware County; Ohio 2023              2355   
...                                                                                ...   
1400000US39159050502 Census Tract 505.02; Union County; Ohio    2023               928   
1400000US39159050601 Census Tract 506.01; Union County; Ohio    2023              4337   
1400000US39159050602 Census Tract 506.02; Union County; Ohio    2023              1995   
1400000US39159050701 Census Tract 507.01; Union County; Ohio    2023              1662   
1400000US39159050702 Census Tract 507.02; Union County; Ohio    2023              1197   

0                                                                                          \
1                                                                                           
2                                                                                Divorced   
GEO_ID               NAME                                       reference_period            
1400000US39041010100 Census Tract 101; Delaware County; Ohio    2023                  170   
1400000US39041010200 Census Tract 102; Delaware County; Ohio    2023                  385   
1400000US39041010420 Census Tract 104.20; Delaware County; Ohio 2023                  144   
1400000US39041010421 Census Tract 104.21; Delaware County; Ohio 2023                   81   
1400000US39041010422 Census Tract 104.22; Delaware County; Ohio 2023                   44   
...                                                                                   ...   
1400000US39159050502 Census Tract 505.02; Union County; Ohio    2023                  132   
1400000US39159050601 Census Tract 506.01; Union County; Ohio    2023                  188   
1400000US39159050602 Census Tract 506.02; Union County; Ohio    2023                  156   
1400000US39159050701 Census Tract 507.01; Union County; Ohio    2023      

Also implemented is the percent table. Which shows the data as a percentage of the total (ie. univierse)

In [16]:
dim_table.percent().T

2025-11-12 11:55:40,392 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:40.247656.wide: Pivoting data into wide format.
2025-11-12 11:55:40,393 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:40.247656.create_description_table: Creating description table from variable labels.
2025-11-12 11:55:40,504 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:40.247656.percent: Creating percent table.


0                                                                                    Total:  \
1                                                                                     Male:   
2                                                                                             
GEO_ID               NAME                                       reference_period              
1400000US39041010100 Census Tract 101; Delaware County; Ohio    2023              50.749465   
1400000US39041010200 Census Tract 102; Delaware County; Ohio    2023              51.172876   
1400000US39041010420 Census Tract 104.20; Delaware County; Ohio 2023              50.757266   
1400000US39041010421 Census Tract 104.21; Delaware County; Ohio 2023              47.410714   
1400000US39041010422 Census Tract 104.22; Delaware County; Ohio 2023              48.606811   
...                                                                                     ...   
1400000US39159050502 Census Tract 505.02; Union County; Ohio    2023              47.371108   
1400000US39159050601 Census Tract 506.01; Union County; Ohio    2023              50.879869   
1400000US39159050602 Census Tract 506.02; Union County; Ohio    2023              54.597701   
1400000US39159050701 Census Tract 507.01; Union County; Ohio    2023              47.772348   
1400000US39159050702 Census Tract 507.02; Union County; Ohio    2023              47.995188   

0                                                                                           \
1                                                                                            
2                                                                                 Divorced   
GEO_ID               NAME                                       reference_period             
1400000US39041010100 Census Tract 101; Delaware County; Ohio    2023              3.640257   
1400000US39041010200 Census Tract 102; Delaware County; Ohio    2023              7.589198   
1400000US39041010420 Census Tract 104.20; Delaware County; Ohio 2023              5.894392   
1400000US39041010421 Census Tract 104.21; Delaware County; Ohio 2023              3.616071   
1400000US39041010422 Census Tract 104.22; Delaware County; Ohio 2023              0.908153   
...                                                                                    ...   
1400000US39159050502 Census Tract 505.02; Union County; Ohio    2023              6.738132   
1400000US39159050601 Census Tract 506.01; Union County; Ohio    2023              2.205537   
1400000US39159050602 Census Tract 506.02; Union County; Ohio    2023              4.269294   
1400000US39159050701 Census Tract 507.01; Union County; Ohio    2023              2.012072   
1400000US39159050702 Census Tract 507.02; Union County; Ohio    2023              2.004812   

0                                                                                            \
1                                                                                   Female:   
2                                                                                             
GEO_ID               NAME                                       reference_period              
1400000US39041010100 Census Tract 101; Delaware County; Ohio    2023              49.250535   
1400000US39041010200 Census Tract 102; Delaware County; Ohio    2023              48.827124   
1400000US39041010420 Census Tract 104.20; Delaware County; Ohio 2023              49.242734   
1400000US39041010421 Census Tract 104.21; Delaware County; Ohio 2023              52.589286   
1400000US39041010422 Census Tract 104.22; Delaware County; Ohio 2023              51.393189   
...                                                                                     ...   
1400000US39159050502 Census Tract 505.02; Union County; Ohio    2023              52.628892   
1400000US39159050601 Census Tract 506.01; Union County; Ohio    2023              49.120131   
1400000US39159050602 Ce

### Timeseries of calls

Dimension table takes any long format table that is formatted similar to the output of CensusAPI. 

This allows for easy implementation of timeseries data by concatenating two long form tables from different years.

In [17]:
b12001_2018 = CensusAPI('acs/acs5', 2018, 'B12001', 'region15', scale='tract', variables=variables)

2025-11-12 11:55:40,528 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2018-tract-region15-b12001.__init__: Initializing CENSUS_API_DATA object for census-acs-acs5-2018-tract-region15-b12001.
2025-11-12 11:55:40,529 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2018/acs/acs5/groups.json with parameters None.
2025-11-12 11:55:40,835 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2018-tract-region15-b12001.validate: Validating selected parameters
2025-11-12 11:55:40,836 | INFO | morpc.census.api.valid_survey_table: acs/acs5 is valid and implemented.
2025-11-12 11:55:40,837 | INFO | morpc.census.api.valid_vintage: 2018 is valid vintage for acs/acs5
2025-11-12 11:55:40,838 | INFO | morpc.req.get_json_safely: Getting data from https://api.census.gov/data/2018/acs/acs5/groups.json with parameters None.
2025-11-12 11:55:41,165 | INFO | morpc.census.api.valid_group: Group B12001 valid group for 2018 acs/acs5.
2025-11-12 11:55:41,167 | INFO | morpc.

In [18]:
import pandas as pd

long = pd.concat([b12001_2023.LONG, b12001_2018.LONG])

In [19]:
dim_table = DimensionTable(long)

2025-11-12 11:55:42,872 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:42.872857.__init__: Initializing DIMENSION_TABLE object.


In [20]:
percent = dim_table.percent()

2025-11-12 11:55:42,887 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:42.872857.wide: Pivoting data into wide format.
2025-11-12 11:55:42,889 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:42.872857.create_description_table: Creating description table from variable labels.
2025-11-12 11:55:43,275 | INFO | morpc.census.api.DimensionTable.2025-11-12 11:55:42.872857.percent: Creating percent table.


In [21]:
percent

GEO_ID                                   1400000US39041010100  \
NAME                  Census Tract 101, Delaware County, Ohio   
reference_period                                         2018   
0     1      2                                                  
Total Male                                          48.549402   
             Divorced                                3.286671   
      Female                                        51.450598   
             Divorced                                6.147292   

GEO_ID                                                         \
NAME                  Census Tract 101; Delaware County; Ohio   
reference_period                                         2023   
0     1      2                                                  
Total Male                                          50.749465   
             Divorced                                3.640257   
      Female                                        49.250535   
             Divorced                                5.139186   

GEO_ID                                   1400000US39041010200  \
NAME                  Census Tract 102, Delaware County, Ohio   
reference_period                                         2018   
0     1      2                                                  
Total Male                                          49.940968   
             Divorced                                5.643447   
      Female                                        50.059032   
             Divorced                                8.642267   

GEO_ID                                                         \
NAME                  Census Tract 102; Delaware County; Ohio   
reference_period                                         2023   
0     1      2                                                  
Total Male                                          51.172876   
             Divorced                                7.589198   
      Female                                        48.827124   
             Divorced                                9.126749   

GEO_ID                                      1400000US39041010420  \
NAME                  Census Tract 104.20, Delaware County, Ohio   
reference_period                                            2018   
0     1      2                                                     
Total Male                                             46.877828   
             Divorced                                   8.190045   
      Female                                           53.122172   
             Divorced                                  11.900452   

GEO_ID                                                            \
NAME                  Census Tract 104.20; Delaware County; Ohio   
reference_period                                            2023   
0     1      2                                                     
Total Male                                             50.757266   
             Divorced                                   5.894392   
      Female                                           49.242734   
             Divorced                                   8.677855   

GEO_ID                                      1400000US39041010421  \
NAME                  Census Tract 104.21, Delaware County, Ohio   
reference_period                                            2018   
0     1      2                                                     
Total Male                                             44.323619   
             Divorced                                   4.915181   
      Female                                           55.676381   
             Divorced                                   7.742497   

GEO_ID                                                            \
NAME                  Census Tract 104.21; Delaware County; Ohio   
reference_period                                            2023   
0     1      2                                                     
Total Mal

### Save the long form table as csv with schema and resource files.

In [22]:
b12001_2023.save('./temp_data')

2025-11-12 11:55:53,195 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.save: Saving data to ./temp_data...
2025-11-12 11:55:53,196 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.save: Writing data to ./temp_data/census-acs-acs5-2023-tract-region15-b12001.long.csv.
2025-11-12 11:55:53,252 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.save: Writing schema to ./temp_data/census-acs-acs5-2023-tract-region15-b12001.schema.yaml.
2025-11-12 11:55:53,467 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.define_schema: Defining schema for B12001 for acs/acs5-year survey in 2023...
2025-11-12 11:55:53,468 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.define_schema: Adding fields for value columns...
2025-11-12 11:55:54,901 | INFO | morpc.census.api.CensusAPI.census-acs-acs5-2023-tract-region15-b12001.define_schema: Schema is valid.
2025-11

In [26]:
import os

print(b12001_2023.FILENAME)

os.path.exists(f'./temp_data/{b12001_2023.FILENAME}')

census-acs-acs5-2023-tract-region15-b12001.long.csv


True

## (DEPRECIATED) Custom API calls

### Getting all available datasets in the api.

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [ ]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [ ]:
from morpc.census.api import get
api = get(url, params)

In [ ]:
api

## Geography tools

In [ ]:
from morpc.census import geos

In [ ]:
geos.SCOPES['region15']

In [ ]:
geos.fetch_geos_from_geoids(['0500000US39049'], '2023', 'ACS')


In [ ]:
from morpc.census.api import geo_params_from_scope_scale
from morpc.census.geos import geoids_from_pseudo, fetch_geos_from_geoids

fetch_geos_from_geoids(geoids_from_pseudo(geo_params_from_scope_scale('region15', 'block group').values()), '2023', 'ACS')

# TODO: Split geoid list into sumlevels and then chucks to avoid to large of requests. Figure out how to handle this in the rest_api module maybe.